In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

from xxchallenge import *

base_height = 35

# Parameters
A = tlm.parameter(-1.50)
G0 = tlm.parameter(-185)
T = tlm.parameter(11)
R = tlm.parameter(4.3)

# XY polynomial parameters
C = tlm.parameter(torch.zeros((13,13), dtype=torch.float64))
fixed_mask = torch.zeros_like(C, dtype=torch.bool)
fixed_mask[0, 0] = True  # Freeze position (0,0)
C.register_hook(lambda grad: grad.masked_fill(fixed_mask, 0.))

cylinder = tlm.ImplicitCylinder(*torch.tensor([-50/2, 50/2, 37.02/2], dtype=torch.float64).unbind())
rod_data = StoreVar(lambda data: data)

# Primary mirror
sag = tlm.SagSum([
    tlm.Parabolic(A=A, normalize=True),
    tlm.XYPolynomial(C, normalize=True)
])
primary = tlm.SagSurface(1800, sag)
primary_data = StoreVar(lambda data: data)

# Optical model
optics = tlm.Sequential(
    tlm.Gap(-1000),
    XXLightSource.load(),

    # Primary mirror
    tlm.Gap(1000-base_height),
    tlm.Translate3D(y=T),
    tlm.Rotate3D(z=R),
    primary_data,
    tlm.ReflectiveSurface(primary),

    # Fixed rod
    tlm.AbsolutePosition(x=G0),
    #tlm.Rotate3D(y=45),
    rod_data,
    NonImagingRod(cylinder),
)

#xxrender(optics, sampling={"xx": 500, "letter": "both"})

In [ ]:
param_groups = [
    {'params': [A], 'lr': 0.03},
    {'params': [G0], 'lr': 100},
    {'params': [T], 'lr': 25},
    {'params': [R], 'lr': 0.05},
    {'params': [C], 'lr': 0.025},
]


record = tlm.optimize(
    optics,
    optimizer = optim.SGD(param_groups),
    sampling = {"xx": 10, "disable_viewer": True, "letter": "positive"},
    dim = 3,
    num_iter = 500,
    nshow=50,
)

plot_record(record, param_groups, optics)
record.best()

print()
print("Final values")
print("A", A)
print("G0", G0)
print("T", T)
print("R", R)

# Print rod position
target = rod_data.value.target()
print("ROD X", target[1].item())
print("ROD Y", target[2].item())
print("ROD Z", -target[0].item())

xxrender(optics, sampling={"xx": 500, "letter": "positive"})

In [ ]:
# ideas for this models:
# regularization so that cube corners at X=-1000 are within the surface
# can be done by collinding 4 rays at the corners
# or just computing F of surface of those points

In [ ]:
# Print rod position
target = rod_data.value.target()
print("ROD POSITION IN XX FRAME")
print("ROD X", target[1].item())
print("ROD Y", target[2].item())
print("ROD Z", -target[0].item())
rod_z = f"z{-target[0].item():.0f}"
print(rod_z)

def xxgridH(N):
    x = np.linspace(-1, 499, N)
    y = np.linspace(-499, 499, N)
    X, Y = np.meshgrid(x, y)
    return np.stack((X, Y), -1).reshape(-1, 2)

with torch.no_grad():
    part_primary = tess_mirror(xxgridH(250), primary_data.value.tf(), primary)

    # flip part around the Y=0 plane to make second part
    part_primary2 = tess_mirror(xxgridH(250), primary_data.value.tf(), primary, flipy=True)

part_sides = makesides(part_primary.vectors.dtype)

mesh.Mesh(np.concatenate([
    part_primary.data,
    part_primary2.data,
    part_sides.data
])).save(f"parabola5-double-{rod_z}.stl")

